### __Pipeline 2: SNP processing with random sampling of the alternative allele__

#### __Requirements__
- a python environment (installed with conda for example);
- .TSV files from the obtained with pipeline 1;
- python `subprocess`;
- python `Numpy`;
- python `Pandas`;

Load some python libraries

In [43]:
import numpy as np
import pandas as pd
from snp_utils import ramdomly_sample_haplotypes, filter_short_introns_from_bed, filter_snps_by_interval, check_snp_codon_length
from sfs_utils import create_unfolded_sfs_from_df, fold_sfs
from mutational_context_utils import *

#### __Import the tables into Pandas__

We are going to use pandas to import the SNP table. Pandas is a great (if used with caution) Python package built on Numpy which allows easy dataFrame manipulations.

In [60]:
%cd ../../masked/vcfs/tables/
!ls 

[Errno 2] No such file or directory: '../../masked/vcfs/tables/'
/Users/tur92196/WorkDir/DGN/dpgp3/masked/vcfs/tables
ZI_chr2L_ann_table.tsv              chr2R_exons_downsampled.pkl
ZI_chr2L_snp_roots.csv              chr2R_exons_imputed.pkl
ZI_chr2L_snp_roots.txt              chr2R_exons_sampled.pkl
ZI_chr2R_ann_table.tsv              chr2R_short_introns_downsampled.pkl
ZI_chr2R_snp_roots.csv              chr2R_short_introns_imputed.pkl
ZI_chr2R_snp_roots.txt              chr2R_short_introns_sampled.pkl
ZI_chr3L_ann_table.tsv              chr3L_exons_downsampled.pkl
ZI_chr3L_snp_roots.csv              chr3L_exons_imputed.pkl
ZI_chr3L_snp_roots.txt              chr3L_exons_sampled.pkl
ZI_chr3R_ann_table.tsv              chr3L_short_introns_downsampled.pkl
ZI_chr3R_snp_roots.csv              chr3L_short_introns_imputed.pkl
ZI_chr3R_snp_roots.txt              chr3L_short_introns_sampled.pkl
chr2L_exons_downsampled.pkl         chr3R_exons_downsampled.pkl
chr2L_exons_imputed.pkl           

/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


Upload the `.tsv` file for each chromosome (except for the __chrom4__ and __chromX__).

In [61]:
# Upload files with pd.read_table()
chr2L_table = pd.read_table("ZI_chr2L_ann_table.tsv")
chr2R_table = pd.read_table("ZI_chr2R_ann_table.tsv")
chr3L_table = pd.read_table("ZI_chr3L_ann_table.tsv")
chr3R_table = pd.read_table("ZI_chr3R_ann_table.tsv")

Check the number of SNPs in each file:

In [62]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_table.shape[0], chr2R_table.shape[0], chr3L_table.shape[0], chr3R_table.shape[0])

'chr2L = 2016260, chr2R = 1582937, chr3L = 1795061, chr3R = 2087683 SNPs!'

In [63]:
# Take a look at chr2L
chr2L_table.head()

,chrom,pos,id,ref,alt,refcount,altcount,refflank,altflank,refcodon,...,snpeff_trnscid,sift_trnscid,sift_geneid,sift_genename,sift_region,sift_vartype,sifts_core,sift_median,sift_pred,deleteriousness
0,chr2L,5090,.,T,C,103,3,CATTTTCTC,CATTCTCTC,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr2L,5095,.,T,A,16,104,TCTCTCCCA,TCTCACCCA,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr2L,5110,.,T,A,130,2,AGGGTGAAA,AGGGAGAAA,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr2L,5118,.,G,T,147,1,ATATGATCG,ATATTATCG,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr2L,5140,.,C,T,158,2,AGTGCCAAC,AGTGTCAAC,NaN,...,FBtr0475186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


UPDATED: 18/MAR/2024
Add a column containing the ancestral allele state. This was obtained by looking at _D. simulans_ genome aligned to _D. melanogaster_ genome. 

In [64]:
# Load each chromsome file with ancestral alleles:
chr2L_aa = pd.read_csv("ZI_chr2L_snp_roots.csv")
chr2R_aa = pd.read_csv("ZI_chr2R_snp_roots.csv")
chr3L_aa = pd.read_csv("ZI_chr3L_snp_roots.csv")
chr3R_aa = pd.read_csv("ZI_chr3R_snp_roots.csv")

# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_aa.shape[0], chr2R_aa.shape[0], chr3L_aa.shape[0], chr3R_aa.shape[0])

'chr2L = 2016260, chr2R = 1582937, chr3L = 1795061, chr3R = 2087683 SNPs!'

In [65]:
tables_dfs = [chr2L_table, chr2R_table, chr3L_table, chr3R_table]
roots_dfs = [chr2L_aa, chr2R_aa, chr3L_aa, chr3R_aa]
for root, table in zip(roots_dfs, tables_dfs):
    if root['POS'].isin(table['pos']).all():
        print("All entries in roots 'pos' column are present in tables 'pos' column.")

        # Add values in roots's "id" column to tables's "id" column
    table['id'] = root['ID']
    print("Updated tables:")
    print(table)
else:
    print("Not all entries in roots 'pos' column are present in tables 'pos' column.")

All entries in roots 'pos' column are present in tables 'pos' column.
Updated tables:
         chrom       pos    id ref alt  refcount  altcount   refflank  \
0        chr2L      5090  AA=T   T   C       103         3  CATTTTCTC   
1        chr2L      5095  AA=T   T   A        16       104  TCTCTCCCA   
2        chr2L      5110  AA=T   T   A       130         2  AGGGTGAAA   
3        chr2L      5118  AA=G   G   T       147         1  ATATGATCG   
4        chr2L      5140  AA=C   C   T       158         2  AGTGCCAAC   
...        ...       ...   ...  ..  ..       ...       ...        ...   
2016255  chr2L  23021415  AA=C   C   A       100         1  AAGCCATAA   
2016256  chr2L  23021467  AA=G   G   C        98         3  ATCTGAGAA   
2016257  chr2L  23021643  AA=T   T   C        48        53  TTATTCTAA   
2016258  chr2L  23021737  AA=T   T   C       100         1  TTAATCGAT   
2016259  chr2L  23022024  AA=T   T   C       100         1  TATATGTAT   

          altflank refcodon  ... snpe

#### __Random sampling of the alternative allele (without rooting)__

Get the total number of haplotypes for each SNP.

In [66]:
# Add a total count column to all datasets:
chr2L_table['totalcount'] = chr2L_table['refcount'] + chr2L_table['altcount']
chr2R_table['totalcount'] = chr2R_table['refcount'] + chr2R_table['altcount']
chr3L_table['totalcount'] = chr3L_table['refcount'] + chr3L_table['altcount']
chr3R_table['totalcount'] = chr3R_table['refcount'] + chr3R_table['altcount']

In [67]:
# Take a look at chr2L
chr2L_table.head()

,chrom,pos,id,ref,alt,refcount,altcount,refflank,altflank,refcodon,...,sift_trnscid,sift_geneid,sift_genename,sift_region,sift_vartype,sifts_core,sift_median,sift_pred,deleteriousness,totalcount
0,chr2L,5090,AA=T,T,C,103,3,CATTTTCTC,CATTCTCTC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106
1,chr2L,5095,AA=T,T,A,16,104,TCTCTCCCA,TCTCACCCA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120
2,chr2L,5110,AA=T,T,A,130,2,AGGGTGAAA,AGGGAGAAA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,132
3,chr2L,5118,AA=G,G,T,147,1,ATATGATCG,ATATTATCG,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148
4,chr2L,5140,AA=C,C,T,158,2,AGTGCCAAC,AGTGTCAAC,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160


Remove SNPs if the totalcount < 160. This defines a lower bound for the random sampling (the minimum number of haplotypes used to sample alternative alleles to).

In [68]:
# Set the lower bound for the number of haplotypes
min_number_of_haplotypes = 160

In [69]:
chr2L_table = chr2L_table[chr2L_table['totalcount'] >= min_number_of_haplotypes]
chr2R_table = chr2R_table[chr2R_table['totalcount'] >= min_number_of_haplotypes]
chr3L_table = chr3L_table[chr3L_table['totalcount'] >= min_number_of_haplotypes]
chr3R_table = chr3R_table[chr3R_table['totalcount'] >= min_number_of_haplotypes]

In [70]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_table.shape[0], chr2R_table.shape[0], chr3L_table.shape[0], chr3R_table.shape[0])

'chr2L = 1919950, chr2R = 1514541, chr3L = 1719317, chr3R = 2017777 SNPs!'

In [75]:
# rooted_indexes = []
# for idx, row in chr2L_table.iterrows():
#     if row['id'].split('=')[1] == row['alt']:
#         rooted_indexes.append(idx)
    

Before perfoming a random sampling on the alternative allele counts, swap the counts between alternative and reference based on the ancestral allele state. If the ancestral is the reference allele, keep as it is; if the ancestral is the alternative allele, make the refence count the alternative allele count.

In [83]:
# Funtion to swap the allele counts of a SNPs given the SNP ancestral state
def root_snp_counts_by_aa(df):
    df_copy = df.copy()
    for idx, row in df_copy.iterrows():
        aa = row['id'].split('=')[1]
        alt = row['alt']
        ref_count = row['refcount']
        alt_count = row['altcount']
        if aa == alt:
            df_copy.loc[idx, 'refcount'], df_copy.loc[idx, 'altcount'] = alt_count, ref_count

    return df_copy

In [92]:
chr2L_table_rooted = root_snp_counts_by_aa(chr2L_table)
chr2R_table_rooted = root_snp_counts_by_aa(chr2R_table)
chr3L_table_rooted = root_snp_counts_by_aa(chr3L_table)
chr3R_table_rooted = root_snp_counts_by_aa(chr3R_table)

Now, random sample the alternative allele count. The function will update the input DataFrame with new values for the total counts, reference, and alternative allele counts

In [93]:
# Impute missing haplotypes
chr2L_table_sampled = ramdomly_sample_haplotypes(chr2L_table_rooted, min_number_of_haplotypes)
chr2R_table_sampled = ramdomly_sample_haplotypes(chr2R_table_rooted, min_number_of_haplotypes)
chr3L_table_sampled = ramdomly_sample_haplotypes(chr3L_table_rooted, min_number_of_haplotypes)
chr3R_table_sampled = ramdomly_sample_haplotypes(chr3R_table_rooted, min_number_of_haplotypes)

#### __Subset SNPs__

__Subset each chromosome to retain only:__
- `SNPs in short-introns`;
- `Synonymous SNPs`;
- `Non-synonymous SNPs`;

__(1) Subset to retain only SNPs annotated as introns__

In [141]:

chr2L_introns = chr2L_table_sampled[(chr2L_table_sampled['effect'] == "INTRON")]
chr2R_introns = chr2R_table_sampled[(chr2R_table_sampled['effect'] == "INTRON")]
chr3L_introns = chr3L_table_sampled[(chr3L_table_sampled['effect'] == "INTRON")]
chr3R_introns = chr3R_table_sampled[(chr3R_table_sampled['effect'] == "INTRON")]


In [142]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_introns.shape[0], chr2R_introns.shape[0], chr3L_introns.shape[0], chr3R_introns.shape[0])

'chr2L = 353457, chr2R = 258584, chr3L = 321868, chr3R = 435809 SNPs!'

__(2)Keep only short intronic SNPs__

For each chromosome, retain only SNPs in `short introns`. To do that, you need to identify the short-intros in Dm6 genome and head/tail the sequence to remove head and trailing 8bp from each short-intron. These extremes migh be under selection constraints. Download the intron regions as `.BED` file from [here](https://genome.ucsc.edu/cgi-bin/hgTables). Select *D. melanogaster* assembly known as *Dm6* (Aug. 2014, BDGP Release 6 + ISO 1 MT/dm6). Define the region of interest as `genome`, select the output format as `BED` and name it. In the next page, select only `Introns plus 0`, then hit `Get BED`. Now you are ready to go. The next step is to create a Pandas DataFrame with short introns intervals.

In [143]:
%cd ../../../../reference/
!ls

/Users/tur92196/WorkDir/DGN/reference
dm3.fa                            dmel-2L-chromosome-r5.57.fasta.gz
dm3.fa.fai                        dmel-2L-r5.57.gff.gz
dm3ToDm6.over.chain               dmel-2R-chromosome-r5.57.fasta.gz
dm6.fa                            dmel-2R-r5.57.gff.gz
dm6.fa.dict                       dmel-3L-chromosome-r5.57.fasta.gz
dm6.fa.fai                        dmel-3L-r5.57.gff.gz
dm6_introns.bed                   dmel-3R-chromosome-r5.57.fasta.gz
dm6_short_introns.bed             dmel-3R-r5.57.gff.gz
dmel-2L-chromosome-r5.57.fasta


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [144]:
short_introns = filter_short_introns_from_bed("dm6_introns.bed", 
                                              "dm6_short_introns.bed", 
                                              ["chr2L", "chr2R", "chr3L", "chr3R"],
                                              short_intron_size=86,
                                              trailling_size=8)
short_introns.head()

,0,1,2,3,4,5
0,chr2L,8387838,8387882,NM_001201797.2_intron_5_0_chr2L_8387832_f,0,+
1,chr2L,8387838,8387882,NM_001201795.2_intron_5_0_chr2L_8387832_f,0,+
2,chr2L,8387838,8387882,NM_164812.5_intron_4_0_chr2L_8387832_f,0,+
3,chr2L,8387838,8387882,NM_205936.3_intron_5_0_chr2L_8387832_f,0,+
4,chr2L,8387838,8387882,NM_205935.3_intron_5_0_chr2L_8387832_f,0,+


Now, apply the filter based on the BED file intervals (to make sure to only keep `short-intronic` SNPs)

In [145]:
# Keep only SNPs in exons and in short-introns
chr2L_short_introns = filter_snps_by_interval(chr2L_introns, short_introns)
chr2R_short_introns = filter_snps_by_interval(chr2R_introns, short_introns)
chr3L_short_introns = filter_snps_by_interval(chr3L_introns, short_introns)
chr3R_short_introns = filter_snps_by_interval(chr3R_introns, short_introns)

Dump these DataFrames as pickle files to avoid re-running it:

In [149]:
# %cd ../dpgp3
#!ls 
%cd WorkDir/DGN/dpgp3/
!ls 

/Users/tur92196/WorkDir/DGN/dpgp3
Popfly                 dmel_data              masked
[delete]dmel_data_bckp dpgp3.txt              originals


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [150]:
# Pickle chrm short-introns DataFrames
# chr2L_short_introns.to_pickle("masked/vcfs/tables/chr2L_short_introns_sampled_rooted.pkl")
# chr2R_short_introns.to_pickle("masked/vcfs/tables/chr2R_short_introns_sampled_rooted.pkl")
# chr3L_short_introns.to_pickle("masked/vcfs/tables/chr3L_short_introns_sampled_rooted.pkl")
# chr3R_short_introns.to_pickle("masked/vcfs/tables/chr3R_short_introns_sampled_rooted.pkl")

In [10]:
# Load pickled chrm short-introns DataFrames
# chr2L_short_introns = pd.read_pickle("masked/vcfs/tables/chr2L_short_introns_sampled_rooted.pkl")
# chr2R_short_introns = pd.read_pickle("masked/vcfs/tables/chr2R_short_introns_sampled_rooted.pkl")
# chr3L_short_introns = pd.read_pickle("masked/vcfs/tables/chr3L_short_introns_sampled_rooted.pkl")
# chr3R_short_introns = pd.read_pickle("masked/vcfs/tables/chr3R_short_introns_sampled_rooted.pkl")

Check how many short-introns SNPs were retained:

In [151]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_short_introns.shape[0], chr2R_short_introns.shape[0], chr3L_short_introns.shape[0], chr3R_short_introns.shape[0])

'chr2L = 2728, chr2R = 2384, chr3L = 2073, chr3R = 2967 SNPs!'

__(3) Take synonymous and non-synonymous SNPs__

In [152]:
chr2L_exons = chr2L_table_sampled[(chr2L_table_sampled['effect'] == "NON_SYNONYMOUS_CODING") | (chr2L_table_sampled['effect'] == "SYNONYMOUS_CODING")]
chr2R_exons = chr2R_table_sampled[(chr2R_table_sampled['effect'] == "NON_SYNONYMOUS_CODING") | (chr2R_table_sampled['effect'] == "SYNONYMOUS_CODING")]
chr3L_exons = chr3L_table_sampled[(chr3L_table_sampled['effect'] == "NON_SYNONYMOUS_CODING") | (chr3L_table_sampled['effect'] == "SYNONYMOUS_CODING")]
chr3R_exons = chr3R_table_sampled[(chr3R_table_sampled['effect'] == "NON_SYNONYMOUS_CODING") | (chr3R_table_sampled['effect'] == "SYNONYMOUS_CODING")]


In [153]:
# Pickle chrm short-introns DataFrames
# chr2L_exons.to_pickle("masked/vcfs/tables/chr2L_exons_sampled_rooted.pkl")
# chr2R_exons.to_pickle("masked/vcfs/tables/chr2R_exons_sampled_rooted.pkl")
# chr3L_exons.to_pickle("masked/vcfs/tables/chr3L_exons_sampled_rooted.pkl")
# chr3R_exons.to_pickle("masked/vcfs/tables/chr3R_exons_sampled_rooted.pkl")

In [12]:
# Load pickled chrm short-introns DataFrames
# chr2L_exons = pd.read_pickle("masked/vcfs/tables/chr2L_exons_sampled_rooted.pkl")
# chr2R_exons = pd.read_pickle("masked/vcfs/tables/chr2R_exons_sampled_rooted.pkl")
# chr3L_exons = pd.read_pickle("masked/vcfs/tables/chr3L_exons_sampled_rooted.pkl")
# chr3R_exons = pd.read_pickle("masked/vcfs/tables/chr3R_exons_sampled_rooted.pkl")

In [154]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_exons.shape[0], chr2R_exons.shape[0], chr3L_exons.shape[0], chr3R_exons.shape[0])

'chr2L = 255680, chr2R = 236781, chr3L = 226182, chr3R = 271278 SNPs!'

#### __Get the SFSs__

For introns:

In [155]:
chr2L_short_introns_sfs = create_unfolded_sfs_from_df(chr2L_short_introns, "altcount", min_number_of_haplotypes)
chr2R_short_introns_sfs = create_unfolded_sfs_from_df(chr2R_short_introns, "altcount", min_number_of_haplotypes)
chr3L_short_introns_sfs = create_unfolded_sfs_from_df(chr3L_short_introns, "altcount", min_number_of_haplotypes)
chr3R_short_introns_sfs = create_unfolded_sfs_from_df(chr3R_short_introns, "altcount", min_number_of_haplotypes)

Then combined each chromosome short-introns SFS

In [156]:
short_introns_sfs_array = np.array([
    chr2L_short_introns_sfs,
    chr2R_short_introns_sfs,
    chr3L_short_introns_sfs,
    chr3R_short_introns_sfs
])

short_introns_sfs = np.sum(short_introns_sfs_array, 0).tolist()

For the synonymous SNPs

In [157]:
chr2L_synonymous = chr2L_exons[(chr2L_exons['effect'] == "SYNONYMOUS_CODING")]
chr2R_synonymous = chr2R_exons[(chr2R_exons['effect'] == "SYNONYMOUS_CODING")]
chr3L_synonymous = chr3L_exons[(chr3L_exons['effect'] == "SYNONYMOUS_CODING")]
chr3R_synonymous = chr3R_exons[(chr3R_exons['effect'] == "SYNONYMOUS_CODING")]

synonymous_sfs_array = np.array([
    create_unfolded_sfs_from_df(chr2L_synonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr2R_synonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr3L_synonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr3R_synonymous, "altcount", min_number_of_haplotypes)
])

synonymous_sfs = np.sum(synonymous_sfs_array, 0).tolist()

And for non-synonymous ones:

In [158]:
chr2L_nonsynonymous = chr2L_exons[(chr2L_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr2R_nonsynonymous = chr2R_exons[(chr2R_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr3L_nonsynonymous = chr3L_exons[(chr3L_exons['effect'] == "NON_SYNONYMOUS_CODING")]
chr3R_nonsynonymous = chr3R_exons[(chr3R_exons['effect'] == "NON_SYNONYMOUS_CODING")]

nonsynonymous_sfs_array = np.array([
    create_unfolded_sfs_from_df(chr2L_nonsynonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr2R_nonsynonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr3L_nonsynonymous, "altcount", min_number_of_haplotypes),
    create_unfolded_sfs_from_df(chr3R_nonsynonymous, "altcount", min_number_of_haplotypes)
])

nonsynonymous_sfs = np.sum(nonsynonymous_sfs_array, 0).tolist()

In [159]:
# Save unfolded SFSs
%cd masked/vcfs/sfss/
!ls

/Users/tur92196/WorkDir/DGN/dpgp3/masked/vcfs/sfss
no-pairing paired


In [160]:
output_sfs_file = "no-pairing/ZI_sfs_si_nopairing_sampled_unfolded_rooted.txt"

with open(output_sfs_file, "w") as of:
    of.write("Introns, synonymous, and nonsynonymous SFS of " + str(min_number_of_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in short_introns_sfs) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in synonymous_sfs) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in nonsynonymous_sfs) + "\n")

#### __Fold the SFSs__

In [161]:
short_introns_sfs_folded = fold_sfs(short_introns_sfs)
synonymous_sfs_folded = fold_sfs(synonymous_sfs)
nonsynonymous_sfs_folded = fold_sfs(nonsynonymous_sfs)

Save the three SFSs to a file:

In [162]:
%cd masked/vcfs/sfss/
!ls 

[Errno 2] No such file or directory: 'masked/vcfs/sfss/'
/Users/tur92196/WorkDir/DGN/dpgp3/masked/vcfs/sfss
no-pairing paired


/opt/homebrew/anaconda3/envs/dmelnexus/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [163]:
output_sfs_file = "no-pairing/ZI_sfs_si_nopairing_sampled_folded_rooted.txt"

with open(output_sfs_file, "w") as of:
    of.write("Introns, synonymous, and nonsynonymous SFS of " + str(min_number_of_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in short_introns_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in synonymous_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in nonsynonymous_sfs_folded) + "\n")

#### __Pair neutral and non-neutral SNPs by mutational context__

Now that we had a simple way to get the SFSs, we are going to add some complexities and pair neutral and non-neutral SNPs by their mutational context.

In [164]:
!ls

no-pairing paired


##### Remove SNPs that are close to each other, as they might cause ambiguity on the sequence category.

For Short-introns:

In [165]:
# Get True for SNPs far apart and False otherwise, and insert a column on each chromosome DataFrame:
# Insert the new column at position 3
chr2L_short_introns.insert(2, "pos_to_keep", find_consecutive_positions(list(chr2L_short_introns["pos"])))
chr2R_short_introns.insert(2, "pos_to_keep", find_consecutive_positions(list(chr2R_short_introns["pos"])))
chr3L_short_introns.insert(2, "pos_to_keep", find_consecutive_positions(list(chr3L_short_introns["pos"])))
chr3R_short_introns.insert(2, "pos_to_keep", find_consecutive_positions(list(chr3R_short_introns["pos"])))

In [166]:
chr2L_short_introns = chr2L_short_introns[chr2L_short_introns['pos_to_keep'] == True]
chr2R_short_introns = chr2R_short_introns[chr2R_short_introns['pos_to_keep'] == True]
chr3L_short_introns = chr3L_short_introns[chr3L_short_introns['pos_to_keep'] == True]
chr3R_short_introns = chr3R_short_introns[chr3R_short_introns['pos_to_keep'] == True]

In [167]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_short_introns.shape[0], chr2R_short_introns.shape[0], chr3L_short_introns.shape[0], chr3R_short_introns.shape[0])

'chr2L = 1850, chr2R = 1632, chr3L = 1484, chr3R = 2134 SNPs!'

For Exons:

In [168]:
# Get True for SNPs far apart and False otherwise, and insert a column on each chromosome DataFrame:
# Insert the new column at position 3
chr2L_exons.insert(2, "pos_to_keep", find_consecutive_positions(list(chr2L_exons["pos"])))
chr2R_exons.insert(2, "pos_to_keep", find_consecutive_positions(list(chr2R_exons["pos"])))
chr3L_exons.insert(2, "pos_to_keep", find_consecutive_positions(list(chr3L_exons["pos"])))
chr3R_exons.insert(2, "pos_to_keep", find_consecutive_positions(list(chr3R_exons["pos"])))

In [169]:
chr2L_exons = chr2L_exons[chr2L_exons['pos_to_keep'] == True]
chr2R_exons = chr2R_exons[chr2R_exons['pos_to_keep'] == True]
chr3L_exons = chr3L_exons[chr3L_exons['pos_to_keep'] == True]
chr3R_exons = chr3R_exons[chr3R_exons['pos_to_keep'] == True]

In [170]:
# Take a look at the number of SNPs with .shape
"chr2L = {}, chr2R = {}, chr3L = {}, chr3R = {} SNPs!".format(chr2L_exons.shape[0], chr2R_exons.shape[0], chr3L_exons.shape[0], chr3R_exons.shape[0])

'chr2L = 222389, chr2R = 209955, chr3L = 199710, chr3R = 240024 SNPs!'

##### Add the sequence category for each SNP

In [171]:
# Insert the new column at position 9
# For introns:
chr2L_short_introns.insert(8, "sequence_category", set_snp_sequence_category(chr2L_short_introns))
chr2R_short_introns.insert(8, "sequence_category", set_snp_sequence_category(chr2R_short_introns))
chr3L_short_introns.insert(8, "sequence_category", set_snp_sequence_category(chr3L_short_introns))
chr3R_short_introns.insert(8, "sequence_category", set_snp_sequence_category(chr3R_short_introns))

# For exons:
chr2L_exons.insert(8, "sequence_category", set_snp_sequence_category(chr2L_exons))
chr2R_exons.insert(8, "sequence_category", set_snp_sequence_category(chr2R_exons))
chr3L_exons.insert(8, "sequence_category", set_snp_sequence_category(chr3L_exons))
chr3R_exons.insert(8, "sequence_category", set_snp_sequence_category(chr3R_exons))

##### Create the sequence category dict

In [172]:
# First create a object for the sequence categories dictionary
sequence_categories_dict = create_sequence_categories_dict()
print(sequence_categories_dict)

{1: ['AA/CA', 'AC/AA'], 2: ['CA/CA', 'CC/AA'], 3: ['GA/CA', 'GC/AA'], 4: ['TA/CA', 'TC/AA'], 5: ['AA/CC', 'AC/AC'], 6: ['CA/CC', 'CC/AC'], 7: ['GA/CC', 'GC/AC'], 8: ['TA/CC', 'TC/AC'], 9: ['AA/CG', 'AC/AG'], 10: ['CA/CG', 'CC/AG'], 11: ['GA/CG', 'GC/AG'], 12: ['TA/CG', 'TC/AG'], 13: ['AA/CT', 'AC/AT'], 14: ['CA/CT', 'CC/AT'], 15: ['GA/CT', 'GC/AT'], 16: ['TA/CT', 'TC/AT'], 17: ['AA/GA', 'AG/AA'], 18: ['CA/GA', 'CG/AA'], 19: ['GA/GA', 'GG/AA'], 20: ['TA/GA', 'TG/AA'], 21: ['AA/GC', 'AG/AC'], 22: ['CA/GC', 'CG/AC'], 23: ['GA/GC', 'GG/AC'], 24: ['TA/GC', 'TG/AC'], 25: ['AA/GG', 'AG/AG'], 26: ['CA/GG', 'CG/AG'], 27: ['GA/GG', 'GG/AG'], 28: ['TA/GG', 'TG/AG'], 29: ['AA/GT', 'AG/AT'], 30: ['CA/GT', 'CG/AT'], 31: ['GA/GT', 'GG/AT'], 32: ['TA/GT', 'TG/AT'], 33: ['AA/TA', 'AT/AA'], 34: ['CA/TA', 'CT/AA'], 35: ['GA/TA', 'GT/AA'], 36: ['TA/TA', 'TT/AA'], 37: ['AA/TC', 'AT/AC'], 38: ['CA/TC', 'CT/AC'], 39: ['GA/TC', 'GT/AC'], 40: ['TA/TC', 'TT/AC'], 41: ['AA/TG', 'AT/AG'], 42: ['CA/TG', 'CT/AG'], 

In [173]:
#  Create three dictionaries for each chromosome
# For introns
chr2L_short_introns_seqclasses = create_snp_dict_wrapper(chr2L_short_introns, sequence_categories_dict, "introns")
chr2R_short_introns_seqclasses = create_snp_dict_wrapper(chr2R_short_introns, sequence_categories_dict, "introns")
chr3L_short_introns_seqclasses = create_snp_dict_wrapper(chr3L_short_introns, sequence_categories_dict, "introns")
chr3R_short_introns_seqclasses = create_snp_dict_wrapper(chr3R_short_introns, sequence_categories_dict, "introns")

# For exons
chr2L_nonsyns_seqclasses, chr2L_syns_seqclasses = create_snp_dict_wrapper(chr2L_exons, sequence_categories_dict, "exons")
chr2R_nonsyns_seqclasses, chr2R_syns_seqclasses = create_snp_dict_wrapper(chr2R_exons, sequence_categories_dict, "exons")
chr3L_nonsyns_seqclasses, chr3L_syns_seqclasses = create_snp_dict_wrapper(chr3L_exons, sequence_categories_dict, "exons")
chr3R_nonsyns_seqclasses, chr3R_syns_seqclasses = create_snp_dict_wrapper(chr3R_exons, sequence_categories_dict, "exons")


##### Find mutational context pairs and get the SFSs
Now, find the pairs and obtain the SFSs for each member of the pair. These are the valid pairs we are looking to have:
- `short-intron SNP` and `non-synonymous SNPs`;
- `short-intron SNP` and `synonymous SNPs`;
- `synonymous SNP` and `non-synonymous SNPs` (maybe?)

For `short-intron SNP` and `non-synonymous SNPs`

In [174]:
# Find pairs of SNPs that are closest to each other
chr2L_si_nonsyn_pairs_si, chr2L_si_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr2L_short_introns_seqclasses, chr2L_nonsyns_seqclasses)
chr2R_si_nonsyn_pairs_si, chr2R_si_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr2R_short_introns_seqclasses, chr2R_nonsyns_seqclasses)
chr3L_si_nonsyn_pairs_si, chr3L_si_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr3L_short_introns_seqclasses, chr3L_nonsyns_seqclasses)
chr3R_si_nonsyn_pairs_si, chr3R_si_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr3R_short_introns_seqclasses, chr3R_nonsyns_seqclasses)

In [175]:
# Get the SFS for the short introns SNPs
# Remember: the data is now sampled, so the max_sample_size means the lower bound set to 160.
# Use the imputed argument, knowing that the data was sampled.
short_introns_paired_nonsyn_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_si_nonsyn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_si_nonsyn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_si_nonsyn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_si_nonsyn_pairs_si, "imputed", min_number_of_haplotypes, folded=True)
])

short_introns_paired_nonsyn_seqclasses_sfs_folded = np.sum(short_introns_paired_nonsyn_sfs_seqclasses_array, 0).tolist()

# Get the SFS for the nonsynonymous SNPs
nonsynonymous_paired_si_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_si_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_si_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_si_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_si_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True)
])

nonsynonymous_paired_si_seqclasses_sfs_folded = np.sum(nonsynonymous_paired_si_sfs_seqclasses_array, 0).tolist()

Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...


In [176]:
# Save the pairs to a file
si_nonsyn_pair_output_sfs_file = "paired/ZI_sfs_si_paired_with_nonsynonymous_sampled_folded_rooted.txt"

with open(si_nonsyn_pair_output_sfs_file, "w") as of:
    of.write("Introns and nonsynonymous SFS of " + str(min_number_of_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in short_introns_paired_nonsyn_seqclasses_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in nonsynonymous_paired_si_seqclasses_sfs_folded) + "\n")

For `short-intron SNP` and `synonymous SNPs`

In [177]:
# Find pairs of SNPs that are closest to each other
chr2L_si_syn_pairs_si, chr2L_si_syn_pairs_syn = find_closest_snp_pairs(chr2L_short_introns_seqclasses, chr2L_syns_seqclasses)
chr2R_si_syn_pairs_si, chr2R_si_syn_pairs_syn = find_closest_snp_pairs(chr2R_short_introns_seqclasses, chr2R_syns_seqclasses)
chr3L_si_syn_pairs_si, chr3L_si_syn_pairs_syn = find_closest_snp_pairs(chr3L_short_introns_seqclasses, chr3L_syns_seqclasses)
chr3R_si_syn_pairs_si, chr3R_si_syn_pairs_syn = find_closest_snp_pairs(chr3R_short_introns_seqclasses, chr3R_syns_seqclasses)

In [178]:
# Get the SFS for the short introns SNPs
short_introns_paired_syn_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_si_syn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_si_syn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_si_syn_pairs_si, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_si_syn_pairs_si, "imputed", min_number_of_haplotypes, folded=True)
])

short_introns_paired_syn_seqclasses_sfs_folded = np.sum(short_introns_paired_syn_sfs_seqclasses_array, 0).tolist()

# Get the SFS for the synonymous SNPs
synonymous_paired_si_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_si_syn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_si_syn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_si_syn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_si_syn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True)
])

synonymous_paired_si_seqclasses_sfs_folded = np.sum(synonymous_paired_si_sfs_seqclasses_array, 0).tolist()

Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...


In [179]:
# Save the pairs to a file
si_syn_pair_output_sfs_file = "paired/ZI_sfs_si_paired_with_synonymous_sampled_folded_rooted.txt"

with open(si_syn_pair_output_sfs_file, "w") as of:
    of.write("Introns and synonymous SFS of " + str(min_number_of_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in short_introns_paired_syn_seqclasses_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in synonymous_paired_si_seqclasses_sfs_folded) + "\n")

For `synonymous SNP` and `non-synonymous SNPs`

In [180]:
# Find pairs of SNPs that are closest to each other
chr2L_syn_nonsyn_pairs_syn, chr2L_syn_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr2L_syns_seqclasses, chr2L_nonsyns_seqclasses)
chr2R_syn_nonsyn_pairs_syn, chr2R_syn_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr2R_syns_seqclasses, chr2R_nonsyns_seqclasses)
chr3L_syn_nonsyn_pairs_syn, chr3L_syn_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr3L_syns_seqclasses, chr3L_nonsyns_seqclasses)
chr3R_syn_nonsyn_pairs_syn, chr3R_syn_nonsyn_pairs_nonsyn = find_closest_snp_pairs(chr3R_syns_seqclasses, chr3R_nonsyns_seqclasses)


In [181]:
# Get the SFS for the synonymous SNPs
synonymous_paired_nonsyn_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_syn_nonsyn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_syn_nonsyn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_syn_nonsyn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_syn_nonsyn_pairs_syn, "imputed", min_number_of_haplotypes, folded=True)
])

synonymous_paired_nonsyn_seqclasses_sfs_folded = np.sum(synonymous_paired_nonsyn_sfs_seqclasses_array, 0).tolist()

# Get the SFS for the nonsynonymous SNPs
nonsynonymous_paired_syn_sfs_seqclasses_array = np.array([
    create_unfolded_sfs_from_snp_dict(chr2L_syn_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr2R_syn_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3L_syn_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True),
    create_unfolded_sfs_from_snp_dict(chr3R_syn_nonsyn_pairs_nonsyn, "imputed", min_number_of_haplotypes, folded=True)
])

nonsynonymous_paired_syn_seqclasses_sfs_folded = np.sum(nonsynonymous_paired_syn_sfs_seqclasses_array, 0).tolist()

Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...
Working with imputed data...


In [183]:
# Save the pairs to a file
syn_nonsyn_pair_output_sfs_file = "paired/ZI_sfs_synonymous_paired_with_nonsynonymous_sampled_folded_rooted.txt"

with open(syn_nonsyn_pair_output_sfs_file, "w") as of:
    of.write("Synonymous and nonsynonymous SFS of " + str(min_number_of_haplotypes) + " samples" + "\n")
    of.write("\t".join(str(item) for item in synonymous_paired_nonsyn_seqclasses_sfs_folded) + "\n")
    of.write("\n")
    of.write("\t".join(str(item) for item in nonsynonymous_paired_syn_seqclasses_sfs_folded) + "\n")